# **Data Engineering for Air Quality Prediction - OpenAQ Air Quality Data Analysis**

**By Rida Shahwar**


This project focuses on **data engineering for air quality analysis** using OpenAQ air quality data. The aim is to preprocess, clean, and engineer features that can be used for predictive modeling. We will:

1. **Collect and load** air quality data.
2. **Clean and preprocess** the data.
3. Perform **feature engineering** on location and time data.
4. Standardize pollutant units and create **pollutant level categories**.
5. Integrate a **model validation** step using Linear and Lasso Regression.


### 1. Data Collection and Loading

In [51]:
import pandas as pd

df = pd.read_excel('openaq (1).xlsx')

In [52]:
df.head()

,Country Code,City,Location,Coordinates,Pollutant,Source Name,Unit,Value,Last Updated,Country Label
0,FR,VERNEUIL,Verneuil,"46.81469300005123, 2.6100689998624795",PM2.5,eea,µg/m³,3.3,2024-10-20T16:00:00-07:00,France
1,FR,TOURS,Pompidou,"47.39463799986039, 0.7048609999996622",PM10,eea,µg/m³,2.3,2024-10-21T17:00:00-07:00,France
2,FR,METZ,Metz- Pont Grilles,"49.12508000010211, 6.181219000343986",NO,eea,µg/m³,25.8,2024-03-11T02:00:00-07:00,France
3,FR,LE LAMENTIN,Lamentin,"14.610653000389345, -61.00258600006397",O3,eea,µg/m³,8.2,2024-10-23T00:00:00-07:00,France
4,FR,LE ROBERT,Robert Bourg,"14.678024999592207, -60.93905599961567",NO2,eea,µg/m³,0.5,2024-10-23T00:00:00-07:00,France


The data contains air quality readings from various locations, with columns including:
- **Country Code**: ISO code for the country.
- **City, Location, Coordinates**: Geographic information.
- **Pollutant, Unit, Value**: Pollutant details.
- **Last Updated**: Timestamp for the reading.

Let's start by cleaning and handling missing values.


### 2. Data Cleaning

#### 2.1 Handling Missing Data and Removing Unecessary Columns

In [53]:
# Check for missing values
print(df.isnull().sum())

# Check for NaN values (NaN is a specific type of missing value)
print(df.isna().sum())

Country Code         0
City             29022
Location             2
Coordinates         70
Pollutant            0
Source Name          0
Unit                 0
Value                0
Last Updated         0
Country Label      127
dtype: int64
Country Code         0
City             29022
Location             2
Coordinates         70
Pollutant            0
Source Name          0
Unit                 0
Value                0
Last Updated         0
Country Label      127
dtype: int64


In [54]:
# Fill missing 'City' with "Unknown"
df['City'] = df['City'].fillna("Unknown")

# Drop rows where 'Location' or 'Coordinates' are missing
df = df.dropna(subset=['Location', 'Coordinates'])

# Drop 'Country Label' column as it's redundant with 'Country Code'
df = df.drop(columns=['Country Label'])

In [55]:
# Check for missing values
print(df.isnull().sum())

# Check for NaN values (NaN is a specific type of missing value)
print(df.isna().sum())

Country Code    0
City            0
Location        0
Coordinates     0
Pollutant       0
Source Name     0
Unit            0
Value           0
Last Updated    0
dtype: int64
Country Code    0
City            0
Location        0
Coordinates     0
Pollutant       0
Source Name     0
Unit            0
Value           0
Last Updated    0
dtype: int64


We handled missing values by:
1. Filling missing values in 'City' with "Unknown".
2. Dropping rows missing 'Location' or 'Coordinates'.
3. Removing the redundant 'Country Label' column.


### 3. Feature Engineering

#### 3.1 Splitting Coordinates into Latitude and Longitude

In [56]:
# Split 'Coordinates' into 'Latitude' and 'Longitude'
df[['Latitude', 'Longitude']] = df['Coordinates'].str.split(',', expand=True)
df['Latitude'] = pd.to_numeric(df['Latitude'])
df['Longitude'] = pd.to_numeric(df['Longitude'])

# Drop the original 'Coordinates' column
df = df.drop(columns=['Coordinates'])

#### 3.2 Date-Time Feature Extraction from Last Updated

In [57]:
# Convert 'Last Updated' to datetime format, ensuring the column type is updated
df['Last Updated'] = pd.to_datetime(df['Last Updated'], errors='coerce', utc=True)
# Force the column to datetime if it's not automatically inferred
if not pd.api.types.is_datetime64_any_dtype(df['Last Updated']):
    df['Last Updated'] = df['Last Updated'].astype('datetime64[ns]') # This line may be redundant after the above change

# Extract date components
df['Year'] = df['Last Updated'].dt.year
df['Month'] = df['Last Updated'].dt.month
df['Day'] = df['Last Updated'].dt.day
df['Hour'] = df['Last Updated'].dt.hour
df['DayOfWeek'] = df['Last Updated'].dt.dayofweek

In [58]:
# Fill NaN values in 'Year' with 0 before converting to int
df['Year'] = df['Year'].fillna(0).astype(int)
df['Month'] = df['Month'].fillna(0).astype(int)  # Convert to int after filling NaN with 0
df['Month'] = df['Month'].apply(lambda x: f"{x:02}") # Format Month column
df['Day'] = df['Day'].fillna(0).astype(int) # Convert to int after filling NaN with 0
df['Day'] = df['Day'].apply(lambda x: f"{x:02}") # Format Day column
df['Hour'] = df['Hour'].fillna(0).astype(int)  # Convert to int after filling NaN with 0
df['Hour'] = df['Hour'].apply(lambda x: f"{x:02}")

In [59]:
# Concatenate into a single column as 'YYYYMMDDHH' and convert to integer
df['DateDecimal'] = (df['Year'].astype(str) +
                     df['Month'] +
                     df['Day'] +
                     df['Hour']).astype(int)

#### 3.3 Standardizing units for pollutant values.

In [60]:
# Define conversion functions for each unit
def convert_to_ug_m3(row):
    if row['Unit'] == 'ppb':
        return row['Value'] * 1.88  # Example conversion factor for NO2
    elif row['Unit'] == 'ppm':
        return row['Value'] * 1880  # Example conversion factor for NO2
    elif row['Unit'] == 'particles/cm³':
        return row['Value'] * 1e6  # Example conversion for particles to µg/m³
    elif row['Unit'] == '%':
        return row['Value'] * 10  # Hypothetical conversion factor for demonstration
    else:
        return row['Value']  # If unit is already µg/m³, return as is

# Apply the function across each row and update the values
df['Value'] = df.apply(convert_to_ug_m3, axis=1)

# Update all units to µg/m³ for consistency
df['Unit'] = 'µg/m³'


In [61]:
df['Unit'].unique()

array(['µg/m³'], dtype=object)

Converted all pollutant values to a common unit ('µg/m³') for consistency.


#### 3.4 Feature Scaling for Value

In [62]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
df['Value_Scaled'] = scaler.fit_transform(df[['Value']])


#### 3.5 Encoding Categorical Variables

In [63]:
df = pd.get_dummies(df, columns=['Country Code', 'Pollutant', 'Source Name'], drop_first=True)

In [64]:
df = pd.get_dummies(df, columns=['City'], prefix='City', drop_first=True)

In [65]:
df = pd.get_dummies(df, columns=['Location'], drop_first=True)

#### 3.6 Pollutant Level Categorization

In [66]:
def categorize_pollution(value):
    if value < 50:
        return 'Low'
    elif 50 <= value < 100:
        return 'Moderate'
    else:
        return 'High'

df['Pollutant_Level'] = df['Value'].apply(categorize_pollution)


In [67]:
df = df.drop(columns=['Last Updated'])

### 4. Model Integration for Validation

In [68]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score


In [69]:
# Check if 'Value_scaled' column exists in the DataFrame
if 'Value_scaled' not in df.columns:
    # If not present, perform scaling here or load data where it is present
    # Example using MinMaxScaler:
    from sklearn.preprocessing import MinMaxScaler
    scaler = MinMaxScaler()
    df['Value_scaled'] = scaler.fit_transform(df[['Value']])

# Now, proceed with dropping the columns
X = df.drop(columns=['Value', 'Value_scaled'])
y = df['Value_scaled']

In [70]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [71]:
# Initialize the model
model = LinearRegression()


In [72]:
# Check for non-numeric columns in X_train
non_numeric_cols = X_train.select_dtypes(include=['object']).columns
print("Non-numeric columns:", non_numeric_cols)

Non-numeric columns: Index(['Unit', 'Month', 'Day', 'Hour', 'Pollutant_Level'], dtype='object')


In [73]:
# Apply one-hot encoding to any remaining categorical columns
X_train = pd.get_dummies(X_train, columns=non_numeric_cols, drop_first=True)
X_test = pd.get_dummies(X_test, columns=non_numeric_cols, drop_first=True)


In [74]:
# Align columns in X_train and X_test
X_train, X_test = X_train.align(X_test, join='left', axis=1, fill_value=0)


In [75]:
# Fit the model on training data
model.fit(X_train, y_train)


LinearRegression()

In [76]:
# Predict on the test set
y_pred = model.predict(X_test)

In [77]:
# Calculate MSE and R-squared
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"R-squared: {r2}")

Mean Squared Error: 2.3403168258915873e-18
R-squared: 0.9999999999999748


In [80]:
import numpy as np
# Drop the 'Unit' column if it's still in X
X = X.select_dtypes(include=[np.number])

from sklearn.linear_model import Lasso
from sklearn.model_selection import cross_val_score

# Initialize Lasso model with a small alpha (regularization strength)
lasso_model = Lasso(alpha=0.1)

# Perform 5-fold cross-validation
cv_mse = cross_val_score(lasso_model, X, y, cv=5, scoring='neg_mean_squared_error')
cv_mse_mean = -cv_mse.mean()  # Convert to positive MSE

print(f"Cross-validated Mean Squared Error: {cv_mse_mean}")



Cross-validated Mean Squared Error: 5.35934557673926e-05
